In [1]:
import duckdb
from pipelines.tasks.config.common import DUCKDB_FILE
from pipelines.tasks.client.https_client import HTTPSClient
from pipelines.tasks.config.common import CACHE_FOLDER
import json
import os
from tqdm import tqdm
import pandas as pd
import geopandas as gpd

In [9]:
# la donnée geo est telechargé depuis https://public.opendatasoft.com/explore/dataset/georef-france-commune/information
https_client = HTTPSClient(
    "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/"
)

In [22]:
path = "georef-france-commune/exports/geojson?lang=fr&timezone=Africa%2FLagos"
filepath = os.path.join(CACHE_FOLDER, "georef-france-commune.geojson")
https_client.download_file_from_https(path, filepath)

Processing file georef-france-commune.geojson: |                                         | 285M/0.00


'georef-france-commune.geojson'

In [2]:
con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [3]:
# Show all tables in the database
query = """
SHOW TABLES;
"""

tables = con.sql(query)
tables_df = tables.df()
tables_df

,name
0,ana__resultats_communes
1,cog_communes
2,edc_communes
3,edc_prelevements
4,edc_resultats
5,laposte_communes
6,mapping_categories
7,stg_communes__cog
8,stg_communes__laposte
9,stg_edc__communes


In [5]:
query_2024 = """
select * from ana__resultats_communes
"""

prelevements_2024 = con.sql(query_2024)
prelevements_2024_df = prelevements_2024.df()
prelevements_2024_df.head(10)

,commune_code_insee,commune_nom,annee,resultat_cvm
0,01001,L'Abergement-Clémenciat,2020,conforme
1,01002,L'Abergement-de-Varey,2020,non analysé
2,01004,Ambérieu-en-Bugey,2020,conforme
3,01005,Ambérieux-en-Dombes,2020,conforme
4,01006,Ambléon,2020,non analysé
5,01007,Ambronay,2020,conforme
6,01008,Ambutrix,2020,conforme
7,01009,Andert-et-Condon,2020,conforme
8,01010,Anglefort,2020,conforme
9,01011,Apremont,2020,conforme


In [13]:
with open(os.path.join(CACHE_FOLDER, "georef-france-commune.geojson"), "r") as file:
    data_geo = json.load(file)
    print("Top-level GeoJSON structure:")
    print("-" * 40)
    for key, value in data_geo.items():
        if key == "features":
            print(f"- {key}: Array with {len(value)} features")
        elif key == "crs":
            print(f"- {key}:")
            print(json.dumps(value, indent=2))
        else:
            print(f"- {key}: {value}")

Top-level GeoJSON structure:
----------------------------------------
- type: FeatureCollection
- features: Array with 34948 features


In [ ]:
def parse_geojson_structure(data, level=0, max_items=1):
    """
    Recursively parse GeoJSON structure and print key-value pairs
    Args:
        data: dictionary or value to parse
        level: current indentation level
        max_items: maximum number of items to show for lists
    """
    indent = "  " * level

    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                print(f"{indent}- {key}:")
                parse_geojson_structure(value, level + 1, max_items)
            else:
                print(f"{indent}- {key}: {value}")

    elif isinstance(data, list):
        print(f"{indent}List with {len(data)} items")
        if data and max_items > 0:
            print(f"{indent}First item:")
            parse_geojson_structure(data[0], level + 1, max_items)


# Read and parse the GeoJSON
with open(os.path.join(CACHE_FOLDER, "georef-france-commune.geojson"), "r") as file:
    data_geo = json.load(file)
    print("GeoJSON Structure:")
    print("-" * 40)
    parse_geojson_structure(data_geo)

In [ ]:
with open(os.path.join(CACHE_FOLDER, "georef-france-commune.geojson"), "r") as file:
    data_geo = json.load(file)
    # Read the GeoJSON file
    # gdf = gpd.read_file(os.path.join(CACHE_FOLDER, "georef-france-commune.geojson"))

    gdf = gpd.read_file(
        os.path.join(CACHE_FOLDER, "georef-france-commune.geojson"),
        dtype={
            "reg_code": str,
            "reg_name": str,
            "dep_code": str,
            "dep_name": str,
            "arrdep_code": str,
            "arrdep_name": str,
            "ze2020_code": str,
            "ze2020_name": str,
            "bv2022_code": str,
            "bv2022_name": str,
            "epci_code": str,
            "epci_name": str,
            "ept_code": str,
            "ept_name": str,
            "com_code": str,
            "com_current_code": str,
            "com_name": str,
        },
    )

    # Print basic information about the GeoJSON
    # print("GeoJSON Info:")
    # print("-" * 40)
    # print(f"Number of features: {len(gdf)}")

In [ ]:
# Print data types of all columns
print("Column Data Types:")
print("-" * 50)
print(gdf.dtypes)

In [ ]:
# For a more detailed view with null counts
print("\nDetailed DataFrame Info:")
print("-" * 50)
gdf.info()

In [ ]:
# Specific information about the geometry column
print("\nGeometry Column Info:")
print("-" * 50)
print(f"Geometry type: {gdf.geometry.geom_type.unique()}")
print(f"CRS: {gdf.crs}")

In [7]:
with open(os.path.join(CACHE_FOLDER, "georef-france-commune.geojson"), "r") as file:
    data_geo = json.load(file)

In [8]:
# Si la solution est validée, il faudra optimiser ce code qui est bien trop lent
data_geo_features = data_geo["features"]
default_properties = {"annee": "2024", "resultat_cvm": "None"}
for i in tqdm(range(len(data_geo_features))):
    elem = data_geo_features[i]
    code_insee = elem["properties"]["com_code"]
    name_insee = elem["properties"]["com_name"]
    if code_insee is not None:
        code_insee = code_insee[0]
        name_insee = name_insee[0]
        prelevement = prelevements_2024_df[
            (prelevements_2024_df.commune_code_insee == code_insee)
        ]
        if len(prelevement) >= 0:
            properties = {}
            properties["commune_code_insee"] = code_insee
            properties["commune_nom"] = name_insee
            for index, row in prelevement.iterrows():
                properties[f"resultat_cvm_{row['annee']}"] = row["resultat_cvm"]
            elem["properties"] = properties
        else:
            # print("elem not found")
            elem["properties"] = default_properties
            elem["properties"]["commune_code_insee"] = code_insee
            elem["properties"]["commune_nom"] = name_insee

100%|██████████| 34948/34948 [07:22<00:00, 79.03it/s]


In [13]:
print(data_geo_features[1])

{'type': 'Feature', 'geometry': {'coordinates': [[[4.564662411513994, 46.99215404485082], [4.564955266861815, 46.99181417997202], [4.565918575437941, 46.99212980012692], [4.566980644093186, 46.992673703487704], [4.568302498730497, 46.99169309012356], [4.568454468451784, 46.991442481159844], [4.568754542311832, 46.99035507679979], [4.568770517378931, 46.989500266284345], [4.568605782581888, 46.98926566922282], [4.568664125299206, 46.98851024019069], [4.568387554182515, 46.987732348464064], [4.568398324766235, 46.986879409143974], [4.568674209861988, 46.98627411059924], [4.569628887000564, 46.985253443229894], [4.569470531458521, 46.985147534977976], [4.565961295179105, 46.98543115617496], [4.566736833870485, 46.984267059929216], [4.567056994428807, 46.983327970665066], [4.567800767517841, 46.98247677830239], [4.569485367211463, 46.97995042011078], [4.573887917095319, 46.972854638411654], [4.574116308745747, 46.9730172185324], [4.574964521631136, 46.97304976563674], [4.576234388550647, 4

In [15]:
new_geo_json = {"type": "FeatureCollection"}
new_geo_json["features"] = data_geo_features

print(new_geo_json["features"][1])

{'type': 'Feature', 'geometry': {'coordinates': [[[4.564662411513994, 46.99215404485082], [4.564955266861815, 46.99181417997202], [4.565918575437941, 46.99212980012692], [4.566980644093186, 46.992673703487704], [4.568302498730497, 46.99169309012356], [4.568454468451784, 46.991442481159844], [4.568754542311832, 46.99035507679979], [4.568770517378931, 46.989500266284345], [4.568605782581888, 46.98926566922282], [4.568664125299206, 46.98851024019069], [4.568387554182515, 46.987732348464064], [4.568398324766235, 46.986879409143974], [4.568674209861988, 46.98627411059924], [4.569628887000564, 46.985253443229894], [4.569470531458521, 46.985147534977976], [4.565961295179105, 46.98543115617496], [4.566736833870485, 46.984267059929216], [4.567056994428807, 46.983327970665066], [4.567800767517841, 46.98247677830239], [4.569485367211463, 46.97995042011078], [4.573887917095319, 46.972854638411654], [4.574116308745747, 46.9730172185324], [4.574964521631136, 46.97304976563674], [4.576234388550647, 4

In [16]:
# Write the dictionary to a GeoJSON file
filename = "georef-france-commune-prelevement.geojson"
write_filepath = os.path.join(CACHE_FOLDER, filename)
with open(
    write_filepath,
    "w",
    encoding="utf-8",
) as file:
    json.dump(new_geo_json, file)

In [17]:
from pipelines.utils.storage_client import ObjectStorageClient
from pipelines.config.config import load_env_variables

load_env_variables()
s3 = ObjectStorageClient()

db_path = DUCKDB_FILE  # Fichier local
s3_path = "dev/geojson/georef-france-commune-prelevement.geojson.removeme"  # Destination sur S3

s3.upload_object(local_path=write_filepath, file_key=s3_path, public_read=True)
print(f"✅ geojson uploadée sur s3://{s3.bucket_name}/{s3_path}")
# La donnée peut maintenant être récupéré depuis https://pollution-eau-s3.s3.fr-par.scw.cloud/dev/geojson/georef-france-commune-prelevement.geojson.removeme

Error upload_object '/Users/jaouadsalahy/Documents/IT/13_pollution_eau/database/cache/georef-france-commune-prelevement.geojson' to bucket 'pollution-eau-s3': Unable to locate credentials


✅ geojson uploadée sur s3://pollution-eau-s3/dev/geojson/georef-france-commune-prelevement.geojson.removeme


# Tests


In [ ]:
df_geo = pd.DataFrame(new_geo_json["features"])
df_geo_flatten = df_geo.join(pd.json_normalize(df_geo["properties"]))
df_geo_flatten

In [ ]:
df_geo_flatten.commune_code_insee.unique()

In [ ]:
# s3.delete_object("dev/geojson/georef-france-commune-prelevement.csv")

In [ ]:
# from pipelines.tasks.config.common import download_file_from_https

# download_file_from_https(
#     url="https://pollution-eau-s3.s3.fr-par.scw.cloud/dev/geojson/georef-france-commune-prelevement-small.geojson.removeme",
#     filepath="test.geojson",
# )

In [ ]:
df_geo = pd.DataFrame(data_geo["features"])
df_geo_flatten = df_geo.join(pd.json_normalize(df_geo["properties"]))
df_geo_flatten.com_code.map(lambda x: len(x) if x is not None else 0).value_counts()

In [ ]:
df_geo_flatten[
    df_geo_flatten.dep_code.map(lambda x: x[0] == "2A" if len(x) == 1 else False)
].com_code